# DESY depth profiling data analysis workbook [2020]

This is a workbook used to integrate synchrotron data from DESY then generate the bg-spline and peak-index files required for running CMWP.

## Import stuff

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import os
import glob
import pyFAI, pyFAI.azimuthalIntegrator
import csv
from shutil import copyfile

from src.cmwp_tools import load_tifs
from src.xrd_tools import getReflections
from src.desy_tools import load_fio
from src.cmwp_tools import getBaseline, getPeaks

## Define settings

In [2]:
### Experiment specific settings

base = "/mnt/manchester_rds/202011_DESY/"                            # This is the base directory where the data is stored

calib_file = base + 'raw/close_detector/calib_new.poni'             # These are the calibration files (.poni) for the detectors

ai = pyFAI.load(calib_file)
wavelength = ai.wavelength * 1e10
print('Wavelength: {0:.8f} A'.format(wavelength))

intpoints = 2000                                                  # This is how many integration points to use for pyFAI

## Zr peaks
peak_name, peak_pos = getReflections(crystalType='hcp', a=3.232, c=5.147, wavelength=0.1848, printReflections=False)

## SPP peaks
spp_peak_name, spp_peak_pos = getReflections(crystalType='hcp', a=5.028, c=8.248, wavelength=0.1848, printReflections=False)
spp_peak_name = spp_peak_name[[5,7,8,9,26]]
spp_peak_pos = spp_peak_pos[[5,7,8,9,26]]

## hydride peaks
delta_peak_name, delta_peak_pos = getReflections(crystalType='fcc', a=4.777, wavelength=0.1848, printReflections=False)
delta_peak_name = delta_peak_name[[0,2]]
delta_peak_pos = delta_peak_pos[[0,2]]

## Background
baseline=[3.1, 3.5, 4.7, 5.2, 5.9, 6.9, 8.8, 9.3, 11.55]             # What 2theta points to calculte the background spline from
baseline_interpolate=[4.1, 7.85, 10.475]                            # What 2theta points to add in extra point
baseline_interpolate_factor=[0.97, 1, 1.15]

limits=[3, 11.6]                                                       # 2theta bounds of integration

searchrange = int(0.005 * intpoints)                         # How many data points each side of the approximate 2theta peak position to search for the true peak

method = 'full_csr'
azimuth_range=None


Wavelength: 0.18505104 A


In [ ]:
### 0.1 dpa hydrided
               
directory = base + "raw/close_detector/hyd/hyd/hyd/"                                  
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd/"       # This is the output directory (normally a sub-folder in your CMWP dir)
fio_file = base + "raw/close_detector/hyd/eh3scan1_00144.fio"
darks = base + "raw/close_detector/hyd/dark"
templates = base + "templates/hyd/"                                      # This is where the template files are stored. These are copied for each integration.
z_range = (14.916, 14.976)
y_range = None
spp=False; hyd=False;

In [ ]:
### 0.1 dpa hydrided **3phase*
               
directory = base + "raw/close_detector/hyd/hyd/hyd/"                                  
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd_3p/"       # This is the output directory (normally a sub-folder in your CMWP dir)
fio_file = base + "raw/close_detector/hyd/eh3scan1_00144.fio"
darks = base + "raw/close_detector/hyd/dark"
templates = base + "templates/hyd_3phase/"                                      # This is where the template files are stored. These are copied for each integration.
z_range = (14.916, 14.976)
y_range = None
spp=True; hyd=True;

In [ ]:
### 0.1 dpa hydrided **3phase* *limazim*
               
directory = base + "raw/close_detector/hyd/hyd/hyd/"                                  
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd_3p_80_100/"       # This is the output directory (normally a sub-folder in your CMWP dir)
fio_file = base + "raw/close_detector/hyd/eh3scan1_00144.fio"
darks = base + "raw/close_detector/hyd/dark"
templates = base + "templates/hyd_3phase_bgfit/"                                      # This is where the template files are stored. These are copied for each integration.
z_range = (14.916, 14.976)
y_range = None
spp=True; hyd=True;

intpoints=1500

method = 'IntegrationMethod(2d int, pseudo split, histogram, cython)'
azimuth_range=[[70,110], [250, 290]]

In [ ]:
### 2 dpa

fio_file = base + "raw/close_detector/dpa2/eh3scan1_00096.fio"
directory = base + "raw/close_detector/dpa2/dpa2/"
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa2/"
darks = base + "raw/close_detector/dpa2/dark"
templates = base + "templates/2dpa/"                                      # This is where the template files are stored. These are copied for each integration.
z_range = (14.830, 14.890)
y_range = None
spp=False; hyd=False;

In [ ]:
### 2 dpa 2p

fio_file = base + "raw/close_detector/dpa2/eh3scan1_00096.fio"
directory = base + "raw/close_detector/dpa2/dpa2/"
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa2_2p/"
darks = base + "raw/close_detector/dpa2/dark"
templates = base + "templates/hyd_3phase_bgfit/"                                         # This is where the template files are stored. These are copied for each integration.
z_range = (14.830, 14.890)
y_range = None
spp=True; hyd=False;

In [ ]:
### 1 dpa

fio_file = base + "raw/close_detector/dpa1/eh3scan1_00077.fio"
directory = base + "raw/close_detector/dpa1/dpa1/"
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa1/"
darks = base + "raw/close_detector/dpa1/dark"
templates = base + "templates/2dpa/"  
z_range = (14.810, 14.870)
y_range = (3.2, 4.3)
spp=False; hyd=False;

In [ ]:
fio_file = base + "raw/close_detector/dpa05/dpa05_02.fio"
directory = base + "raw/close_detector/dpa05/dpa05_02/"
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa05/"
darks = base + "raw/close_detector/dpa05/dark"
templates = base + "templates/2dpa/"   
z_range = (14.780, 14.860)
y_range = None
spp=False; hyd=False;

In [31]:
fio_file = base + "raw/close_detector/dpa05/dpa05_02.fio"
directory = base + "raw/close_detector/dpa05/dpa05_02/"
outputdir = "/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa05_2p/"
darks = base + "raw/close_detector/dpa05/dark"
templates = base + "templates/hyd_3phase_bgfit/"
z_range = (14.780, 14.860)
y_range = (-5.7, -3.7)
spp=True; hyd=False;

In [ ]:
## Integrate lab6

In [ ]:
#lab6 = glob.glob('/mnt/manchester_rds/202011_DESY/raw/close_detector/LaB6/LaB6_00*.tif')[2:]
#lab6_dark = glob.glob('/mnt/manchester_rds/202011_DESY/raw/close_detector/LaB6/dark*.tif')

#ai = pyFAI.load(calib_file)

#data=load_tifs(lab6)-load_tifs(lab6_dark)

In [ ]:
##output = ai.integrate1d(data, npt=intpoints, radial_range=[2.2,11.3],
        correctSolidAngle=True, method='full_csr',polarization_factor=0.99, unit='2th_deg')
#np.savetxt('/mnt/manchester_rds/202011_DESY/raw/close_detector/LaB6.dat', np.array(output).T)

In [ ]:
#output = ai.integrate1d(data, npt=intpoints, radial_range=[2.2,11.3], azimuth_range=[90,110],
#        method='IntegrationMethod(2d int, pseudo split, histogram, cython)', correctSolidAngle=True, polarization_factor=0.99, unit='2th_deg')
#np.savetxt('/mnt/manchester_rds/202011_DESY/raw/close_detector/LaB6_limazim.dat', np.array(output).T)

## Read log file and make pandas table

The .fio file contains a table of motor positions (ie idtz2, idty1), image names and whether the image was a clearing frame or actual exposure. The section reads in the file into a Pandas dataframe for ease of use.

In [32]:
df = load_fio(filename=fio_file)

if z_range is not None:
    df = df[np.logical_and(df['idtz2']>z_range[0], df['idtz2']<z_range[1])]
if y_range is not None:
    df = df[(df['idty2(encoder)']>y_range[0]) & (df['idty2(encoder)']<y_range[1])]

df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [33]:
# Print the dataframe
df

,idty2(encoder),end pos,idtz2,channel,filename,type,unix time
0,-4.99994,-4.99994,14.782,2,dpa05_00282.cbf,exposure,1.605998e+09
1,-4.49994,-4.49994,14.782,2,dpa05_00283.cbf,exposure,1.605998e+09
2,-3.99994,-3.99994,14.782,2,dpa05_00284.cbf,exposure,1.605998e+09
3,-3.99994,-3.99994,14.784,2,dpa05_00287.cbf,exposure,1.605998e+09
4,-4.49994,-4.49994,14.784,2,dpa05_00288.cbf,exposure,1.605998e+09
...,...,...,...,...,...,...,...
131,-4.99994,-4.99994,14.856,2,dpa05_00469.cbf,exposure,1.605998e+09
132,-5.49994,-5.49994,14.856,2,dpa05_00470.cbf,exposure,1.605998e+09
133,-4.99994,-4.99994,14.858,2,dpa05_00472.cbf,exposure,1.605999e+09
134,-4.49994,-4.49994,14.858,2,dpa05_00473.cbf,exposure,1.605999e+09


In [34]:
#df = df.iloc[100:102]

## Integration, bg-spline and peak-index creation

This section takes all the images in the above table and integrates them according to the calibrations defined previously for all 4 detectors. 
This is saved as a .dat file with a prefix containing the motor positions. Then, the files in the template directory are copied with the same prefix. 
A background spline is created from the baseline points specified above and saved with the .bg-spline.dat suffix.
The a peak-index.dat file is made based on the Zr indexes specifed above.

In [35]:
# Get list of unique motor position values
z_values = df['idtz2'].unique()
y_values = df['idty2(encoder)'].unique()

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
if not os.path.exists(outputdir + '/0plots'):
    os.makedirs(outputdir + '/0plots')
    
pe1_darks_list = glob.glob(darks + '/*.cbf')[2:]
dark = load_tifs(pe1_darks_list)

print('z from {0:.3f} to {1:.3f}'.format(np.min(z_values), np.max(z_values)))
print('y from {0:.3f} to {1:.3f}'.format(np.min(y_values), np.max(y_values)))

int_list = []
num_error = 0

for index, row in df.iterrows():
    y=row['idty2(encoder)']
    z=row['idtz2']
    
    prefix = 'y_{1:.3f}_z_{0:.3f}'.format(z,y)

    file = (df[(df['idtz2']==z) & (df['idty2(encoder)']==y)])['filename'].values[0]
    print('\rCurrent:   {0} / {1}\ty = {2:.3f} / {3:.3f}\tz = {4:.3f} / {5:.3f}     [{6}]       {7} file error(s)       '
          .format(index+1, len(df.index), y, np.max(y_values), z, np.max(z_values), file, num_error), end='')

    file = directory + file

    ################### Integrate #############################################

    try:
        data = load_tifs(file) - dark
    except:
        num_error +=1
    else:
        outputs = []
        if azimuth_range == None:
            output = ai.integrate1d(data, npt=intpoints, correctSolidAngle=True, method=method, 
                                    polarization_factor=0.99, unit='2th_deg', radial_range=(limits[0], limits[1]))
            xvals = output[0]
            yvals = output[1]
            
        else:
            ## If azimuth range is specified
            for azim in azimuth_range:
                outputs.append(ai.integrate1d(data, npt=intpoints, correctSolidAngle=True, method=method, azimuth_range=azim,
                                    polarization_factor=0.99, unit='2th_deg', radial_range=(limits[0], limits[1])))

            xvals=outputs[0][0]; 
            yvals=np.sum([output[1] for output in outputs], axis=0); 
        
        yvals = (yvals - np.min(yvals)) + 20

        for templateName in glob.glob(templates + '*'):
            copyfile(templateName, outputdir + prefix + templateName.split('/')[-1][8:])

        ####################### Save integrated data ######################

        with open(outputdir + prefix + '.dat', 'w+') as f:
             np.savetxt(fname = f, X=np.transpose([xvals, yvals]), fmt = ('%1.5f'))

        ########################### Make figure ###########################

        plt.ioff()
        fig, (ax2) = plt.subplots(1, 1, figsize=(16,8))
        ax2.set_title('Integrated data');
        ax2.set_xlabel('2theta (deg)'); 
        ax2.set_ylabel('Intensity');
        ax2.plot(xvals, yvals)
        ax2.set_xlim(np.min(xvals)+0.01, np.max(xvals)-0.01)

        ########################### Make bg-spline ###########################
      
        baseline_pos, baseline_int, cs = getBaseline(xvals, yvals, baseline, baseline_interpolate, baseline_interpolate_factor,
                                             write_to = outputdir + prefix + '.bg-spline.dat')

        ax2.plot(xvals, cs(xvals))
        ax2.plot(baseline_pos, baseline_int, 'o',c='r')

        ########################### Make Zr peak-index ###########################

        peak_pos_new, peak_name_new, peak_int_new = getPeaks(xvals, yvals, 
                                             peak_pos, peak_name, cs, searchrange, 
                                                    ax=ax2, plotcolour='r',
                                             write_to = outputdir + prefix + '.peak-index.dat')
        
        ########################
        if spp is True:

            peak_pos_new, peak_name_new, peak_int_new = getPeaks(xvals, yvals, 
                                                 spp_peak_pos, spp_peak_name, cs, searchrange, 
                                                    mode='a', phasenumber=2, 
                                                    ax=ax2, plotcolour='g',
                                                 write_to = outputdir + prefix + '.peak-index.dat')
            peak_pos_new, peak_name_new, peak_int_new = getPeaks(xvals, yvals, 
                                                 np.array([4.1956]), np.array(['110']), cs, 1, 
                                                    mode='a', phasenumber=2, 
                                                ax=ax2, plotcolour='g',
                                                 write_to = outputdir + prefix + '.peak-index.dat')
        if hyd is True:
            peak_pos_new, peak_name_new, peak_int_new = getPeaks(xvals, yvals, 
                                                 delta_peak_pos, delta_peak_name, cs, 3, 
                                                    mode='a', phasenumber=3, 
                                                ax=ax2, plotcolour='k',
                                                 write_to = outputdir + prefix + '.peak-index.dat')
        ########################
        
    #Append to list
    int_list.append(np.sum(yvals - cs(xvals)))

    ## Save plots #################################

    plt.savefig(outputdir + '/0plots/plot_' + prefix + '.pdf')
    plt.yscale('log')
    plt.savefig(outputdir + '/0plots/log_' + prefix + '.pdf')
    plt.close()


z from 14.782 to 14.858
y from -5.500 to -4.000
Current:   136 / 136	y = -4.000 / -4.000	z = 14.858 / 14.858     [dpa05_00474.cbf]       1 file error(s)       

## Plot integrated intensities

In [ ]:
plt.close()

for val in df['idty2(encoder)'].unique():
    yval = np.array(int_list)[np.array(df['idty2(encoder)'].tolist() == val)]
    zval = np.array(df['idtz2'].tolist())[np.array(df['idty2(encoder)'].tolist() == val)]
    
    # Take differential and calculate edge position
    edge = zval[np.argmax(np.gradient(yval))]
    
    plt.plot(zval, yval, label = 'y = {0:.3f}   edge = {1:.3f}'.format(val, edge))
    
plt.xlabel('Z position (mm)')
plt.ylabel('Integrated intensity')
plt.show()
plt.legend(loc='lower right')

In [ ]:
plt.savefig(outputdir + '/0plots/0integrated_intensity.pdf')

## Make a bash script

Save this text into a .sh file and run it - this will execute CMWP for each file sequentially

In [ ]:
cmwpfolder = "/home/rhys/Documents/CMWP-211102/"

for index, row in df.iterrows():
    y=row['idty2(encoder)']; z=row['idtz2'];    
    
    print('./evaluate ' + outputdir.split(cmwpfolder)[-1] + 'y_{1:.3f}_z_{0:.3f}.dat auto'.format(z,y))

## 2D integration

In [ ]:
data = [load_tifs(file) for file in files]

output2d = ais.integrate2d(data, npt_rad=intpoints, npt_azim=3600, correctSolidAngle=True, polarization_factor=0.99)
output1d = ais.integrate1d(data, npt=intpoints, correctSolidAngle=True, polarization_factor=0.99)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(14,12))
ax1.imshow(np.where(output2d[0]==0, np.nan, output2d[0]), vmin=0, vmax=0.5e11)

ax2.plot(output1d[0], output1d[1])
ax2.set_xlim(np.min(output1d[0]),np.max(output1d[0]))
ax2.set_yscale('log')

for name, pos in zip(peak_name, peak_pos):
    
    if np.min(output1d[0]) < pos < np.max(output1d[0]):

        frac = 1.42*intpoints*(pos-np.min(output1d[0]))/(np.max(output1d[0]))
        
        # draw line and print name
        ax2.axvline(pos, alpha=0.3, c='r')
        ax1.axvline(frac, alpha=0.3, c='r')
        
        ax1.text(frac, 3500, name, horizontalalignment = 'center', c='r')
        ax2.text(pos, np.max(output1d[1]), name, horizontalalignment = 'center', c='r')


plt.show()